<a href="https://colab.research.google.com/github/albertbaezd/data-mining-final-project/blob/develop/Data_Mining_project_code_Albert_Random.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import drive
from sklearn.utils import resample
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier


# import the datasets: -  - Albert: remove comment if you want to manually place the files here. If not, it's going to use a Google Drive folder
# train_data = pd.read_csv("census-income.data.csv") - Albert: remove comment if you want to manually place the files here. If not, it's going to use a Google Drive folder
# test_data = pd.read_csv("census-income.test.csv")  - Albert: remove comment if you want to manually place the files here. If not, it's going to use a Google Drive folder



In [ ]:
'''
Here, we authenticate with Google Drive to add the datafiles from a Drive folder

Instructions:

1. Create a folder in your Google Drive, called data-mining-csv-files
2. Insert both csv files (test and train) with the original names there.
3. Just run this cell to import the code
'''
from google.colab import drive
drive.mount('/content/drive')

train_data = pd.read_csv("/content/drive/My Drive/data-mining-csv-files/census-income.data.csv") #- Albert: remove comment if you want to manually place the files here. If not, it's going to use a Google Drive folder
test_data = pd.read_csv("/content/drive/My Drive/data-mining-csv-files/census-income.test.csv")  #- Albert: remove comment if you want to manually place the files here. If not, it's going to use a Google Drive folder

# Displaying results:

print(train_data)
print(test_data)

Mounted at /content/drive
       age         work-class  fnlwgt    education  education_num  \
0       39          State-gov   77516    Bachelors             13   
1       50   Self-emp-not-inc   83311    Bachelors             13   
2       38            Private  215646      HS-grad              9   
3       53            Private  234721         11th              7   
4       28            Private  338409    Bachelors             13   
...    ...                ...     ...          ...            ...   
32556   27            Private  257302   Assoc-acdm             12   
32557   40            Private  154374      HS-grad              9   
32558   58            Private  151910      HS-grad              9   
32559   22            Private  201490      HS-grad              9   
32560   52       Self-emp-inc  287927      HS-grad              9   

            marital-status          occupation    relationship    race  \
0            Never-married        Adm-clerical   Not-in-family   White 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE


# Assuming train_data and test_data are already loaded as dataframes
# train_data, test_data

# Function to preprocess data

def preprocess_data(data, transformer=None, is_train=True):
    print("Initial columns:", data.columns)  # Initial check of all columns in the dataset

    income_column = 'income'
    if income_column not in data.columns:
        raise ValueError(f"The column '{income_column}' does not exist in the dataset.")

    # If it's test data, handle the period at the end
    if not is_train:
        data[income_column] = data[income_column].str.strip().str.replace('.', '')
        print("Adjusted 'income' for test data:", data[income_column].unique())  # Debug

    # Identify and store categorical features excluding 'income'
    categorical_features = data.select_dtypes(include=['object']).columns.tolist()
    if income_column in categorical_features:
        categorical_features.remove(income_column)
    print("Categorical features before encoding:", categorical_features)  # Debug statement

    # Apply ColumnTransformer only to categorical features excluding 'income'
    if transformer is None:
        transformer = ColumnTransformer(
            transformers=[('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)],
            remainder='passthrough')
        X = transformer.fit_transform(data.drop(columns=[income_column]))
    else:
        X = transformer.transform(data.drop(columns=[income_column]))

    # Now encode the 'income' column after categorical handling
    y = LabelEncoder().fit_transform(data[income_column])
    print("Encoded 'income':", np.unique(y))  # Debug the output of the encoded income

    return X, y, transformer


# Function to train the Random Forest model with SMOTE
def train_random_forest_with_smote(X, y):
    # Apply SMOTE
    smote = SMOTE(random_state=42)
    X_smote, y_smote = smote.fit_resample(X, y)

    # Train the Random Forest
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_smote, y_smote)

    return model

# Evaluate the model
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)

    return accuracy, report

# Main execution function
def main():
    # Preprocess training data
    X_train, y_train, transformer = preprocess_data(train_data, is_train=True)

    # Train model with SMOTE
    smote = SMOTE(random_state=42)
    X_smote, y_smote = smote.fit_resample(X_train, y_train)
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_smote, y_smote)

    # Preprocess testing data using the same transformer
    X_test, y_test, _ = preprocess_data(test_data, transformer, is_train=False)

    # Evaluate the model
    accuracy, report = evaluate_model(model, X_test, y_test)
    print(f'Accuracy: {accuracy}')
    print(f'Classification Report:\n{report}')

if __name__ == '__main__':
    main()

Initial columns: Index(['age', 'work-class', 'fnlwgt', 'education', 'education_num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-worked-per-week',
       'native-country', 'income'],
      dtype='object')
Categorical features before encoding: ['work-class', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
Encoded 'income': [0 1]
Initial columns: Index(['age', 'work-class', 'fnlwgt', 'education', 'education_num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-worked-per-week',
       'native-country', 'income'],
      dtype='object')
Adjusted 'income' for test data: ['<=50K' '>50K']
Categorical features before encoding: ['work-class', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']
Encoded 'income': [0 1]
Accuracy: 0.8487193661323015
Classification Report:
  

In [ ]:
train_data.columns

Index(['age', 'work-class', 'fnlwgt', 'education', 'education_num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-worked-per-week',
       'native-country', 'income'],
      dtype='object')

In [ ]:
'''
Random Forest implementation
'''

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Step 1 - declaring x and y values for random forests

x_train_rf = train_data.drop('income', axis=1)  # Features
y_train_rf = train_data['income']               # Target variable

x_test_rf = test_data.drop('income', axis=1)  # Features
y_test_rf = test_data['income']               # Target variable

# Step 2:
# Perform one-hot encoding if there are categorical variables
# Example:

'''
OHE = One Hot Encoding. It's required to run the Random Forest Classifier from SKLEARN
'''

x_train_rf_ohe = pd.get_dummies(x_train_rf)
x_test_rf_ohe = pd.get_dummies(x_test_rf)

# Step 3: Instantiate the Random Forest Classifier
# You can specify hyperparameters such as n_estimators, max_depth, etc.
# Example:
# rf_classifier = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf_classifier_train = RandomForestClassifier(random_state=42)
rf_classifier_test = RandomForestClassifier(random_state=42)

# Step 4: Train the models
rf_classifier_train.fit(x_train_rf_ohe, y_train_rf)
rf_classifier_test.fit(x_test_rf_ohe, y_test_rf)

# Step 5: Making predictions - Assuming 'new_data' is the new DataFrame
# we want to check containing features (excluding the income column)

# x_train_rf = train_data.drop('income', axis=1)  # Features
# y_train_rf = train_data['income']               # Target variable
predictions_train_rf = rf_classifier_train.predict(x_train_rf_ohe)

predictions_test_rf = rf_classifier_test.predict(x_test_rf_ohe)

print("Predictions in training \n")
print(predictions_train_rf)

print("Predictions in TESTING \n")
print(predictions_test_rf)



Predictions in training 

[' <=50K' ' <=50K' ' <=50K' ... ' <=50K' ' <=50K' ' >50K']
Predictions in TESTING 

[' <=50K.' ' <=50K.' ' >50K.' ... ' <=50K.' ' <=50K.' ' >50K.']


In [ ]:
'''
Confusion matrix for Random Forests
'''

from sklearn.metrics import confusion_matrix, accuracy_score

# Calculate confusion matrix
conf_matrix_rf = confusion_matrix(y_test_rf, predictions_test_rf)

# Calculate accuracy
accuracy_rf = accuracy_score(y_test_rf, predictions_test_rf)

print("Confusion Matrix:")
print(conf_matrix_rf)
print("Accuracy:", accuracy_rf)


Confusion Matrix:
[[12435     0]
 [    1  3845]]
Accuracy: 0.9999385787113814


In [ ]:
'''
Code block to try to replace missing values
'''

# This function takes a dataframe and returns * A MODIFIED COPY *. Doesn't change the original
def replace_question_marks(df, columns_to_replace):
    most_common_elements = {}
    most_common_counts = {}
    df_copy = df.copy()  # Create a copy of the original DataFrame

    for col in columns_to_replace:
        # Find the most common element and its count in the column
        value_counts = df[col].value_counts()
        most_common = value_counts.idxmax()
        most_common_count = value_counts.max()

        most_common_elements[col] = most_common
        most_common_counts[col] = most_common_count

        # Replace ' ?' with the most common element in the column
        df_copy[col] = df_copy[col].replace(' ?', most_common)

    return df_copy, most_common_elements, most_common_counts

columns_to_replace = ['work-class', 'occupation', 'native-country']



In [ ]:
'''
Testing without missing values
'''
data_train_modified_rf, most_common_elements_data_train_rf, most_common_counts_data_train_rf = replace_question_marks(train_data, columns_to_replace)
data_test_modified_rf, most_common_elements_data_test_rf, most_common_counts_data_test_rf = replace_question_marks(test_data, columns_to_replace)

# Step 1 - declaring x and y values for random forests

x_train_rf_modified = data_train_modified_rf.drop('income', axis=1)  # Features
y_train_rf_modified = data_train_modified_rf['income']               # Target variable

x_test_rf_modified = data_test_modified_rf.drop('income', axis=1)  # Features
y_test_rf_modified = data_test_modified_rf['income']               # Target variable

# Step 2:
# Perform one-hot encoding if there are categorical variables
# Example:

'''
OHE = One Hot Encoding. It's required to run the Random Forest Classifier from SKLEARN
'''

x_train_rf_ohe_modified = pd.get_dummies(x_train_rf_modified)
x_test_rf_ohe_modified = pd.get_dummies(x_test_rf_modified)

# Step 3: Instantiate the Random Forest Classifier
# You can specify hyperparameters such as n_estimators, max_depth, etc.
# Example:
# rf_classifier = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf_classifier_train1 = RandomForestClassifier(random_state=42)
rf_classifier_test1 = RandomForestClassifier(random_state=42)

# Step 4: Train the models
rf_classifier_train1.fit(x_train_rf_ohe_modified, y_train_rf_modified)
rf_classifier_test1.fit(x_test_rf_ohe_modified, y_test_rf_modified)

# Step 5: Making predictions - Assuming 'new_data' is the new DataFrame
# we want to check containing features (excluding the income column)

# x_train_rf = train_data.drop('income', axis=1)  # Features
# y_train_rf = train_data['income']               # Target variable
predictions_train_rf1 = rf_classifier_train1.predict(x_train_rf_ohe_modified)

predictions_test_rf1 = rf_classifier_test1.predict(x_test_rf_ohe_modified)

print("Predictions in training \n")
print(predictions_train_rf1)

print("Predictions in TESTING \n")
print(predictions_test_rf1)



Predictions in training 

[' <=50K' ' <=50K' ' <=50K' ... ' <=50K' ' <=50K' ' >50K']
Predictions in TESTING 

[' <=50K.' ' <=50K.' ' >50K.' ... ' <=50K.' ' <=50K.' ' >50K.']


In [ ]:
'''
Confusion matrix for Random Forests # Without missing
'''

from sklearn.metrics import confusion_matrix, accuracy_score

# Calculate confusion matrix
conf_matrix_rf1 = confusion_matrix(y_test_rf_modified, predictions_test_rf1)

# Calculate accuracy
accuracy_rf1 = accuracy_score(y_test_rf_modified, predictions_test_rf1)

print("Confusion Matrix new:")
print(conf_matrix_rf1)
print("Accuracy new:", accuracy_rf1)


Confusion Matrix new:
[[12435     0]
 [    1  3845]]
Accuracy new: 0.9999385787113814
